# Quickstart

Get started here. We will assess a payment default prediction model for gender fairness using Lens, in 5 minutes. More in-depth information can be found in the [lens FAQ](https://credoai-lens.readthedocs.io/en/latest/notebooks/lens_faq.html#How-can-I-choose-which-assessments-to-run?)

**Setup**

Lens installation instruction can be found on [readthedocs](https://credoai-lens.readthedocs.io/en/stable/setup.html)

**Find the code**

Download this notebook [from github.](https://raw.githubusercontent.com/credo-ai/credoai_lens/develop/docs/notebooks/quickstart.ipynb).

### Get your ML environment ready
 In this tutorial we will emulate the modeling phase by running a quick script. This script loads a dataset, splits it into training and testing, and fits a model. You can see the full script [here](https://github.com/credo-ai/credoai_lens/blob/release/1.0.0/docs/notebooks/training_script.py).

Here we have a gradient boosted classifier trained on the UCI Credit Card Default Dataset.

In [1]:
# model and X_test, y_test, etc. are defined by this script
%run training_script.py

/opt/homebrew/Caskroom/miniforge/base/envs/dev/lib/python3.9/site-packages/huggingface_hub/snapshot_download.py:6: FutureWarning: snapshot_download.py has been made private and will no longer be available from version 0.11. Please use `from huggingface_hub import snapshot_download` to import the only public function in this module. Other members of the file may be changed without a deprecation notice.
  warnings.warn(


### Imports

In [2]:
# Import Lens and necessary artifacts
from credoai.lens import Lens
from credoai.artifacts import ClassificationModel, TabularData

In lens, the classes that evaluate models and/or datasets are called `evaluators`. In this example we are interested in evaluating the model's fairness. For this we can use the `ModelFairness` evaluator. We'll
also evaluate the model's performance.

In [3]:
from credoai.evaluators import ModelFairness, Performance

## Lens in 5 minutes

Below is a basic example where our goal is to evaluate the above model. We'll break down this code [below](#Breaking-Down-The-Steps).

Briefly, the code is doing four things:

* Wrapping ML artifacts (like models and data) in Lens objects
* Initializing an instance of Lens. Lens is the main object that performs evaluations. Under the hood, it creates a `pipeline` of evaluations that are run.
* Add evaluators to Lens.
* Run Lens

In [4]:
# set up model and data artifacts
credo_model = ClassificationModel(name="credit_default_classifier", model_like=model)
credo_data = TabularData(
    name="UCI-credit-default",
    X=X_test,
    y=y_test,
    sensitive_features=sensitive_features_test,
)

# Initialization of the Lens object
lens = Lens(model=credo_model, assessment_data=credo_data)

# initialize the evaluator and add it to Lens
metrics = ['precision_score', 'recall_score', 'equal_opportunity']
lens.add(ModelFairness(metrics=metrics), id='MyModelFairness')
lens.add(Performance(metrics=metrics), id='MyModelPerformance')

# run Lens
lens.run()
lens.get_results()['MyModelFairness'][0]

2022-09-28 18:31:13,560 - lens - INFO - Evaluator ModelFairness added to pipeline. Dataset used: assessment_data. Sensitive feature: SEX
2022-09-28 18:31:13,759 - lens - INFO - fairness metric, equal_opportunity, unused by PerformanceModule
2022-09-28 18:31:13,782 - lens - INFO - Evaluator Performance added to pipeline. 
2022-09-28 18:31:13,782 - lens - INFO - Running evaluation for step: MyModelFairness
2022-09-28 18:31:13,806 - lens - INFO - Running evaluation for step: MyModelPerformance


,type,value,subtype
0,equal_opportunity,0.027686,fairness
1,precision_score,0.016322,parity
2,recall_score,0.027686,parity


`lens.get_results()` provides a dictionary where the results of the evaluators are stored as values, and the keys correspond to the ids of the evaluators.  

In the previous case we specified the id of the evaluator when we added `ModelFairness` to the pipeline, however `id` is an optional argument for the `add` method. If omitted, a random one will be generated.

## Using Len's pipeline argument

If we want to add multiple evaluators to our pipeline, one way of doing it could be repeating the `add` step, as shown above. Another way is to define the pipeline steps, and pass it to `Lens` at initialization time. Let's explore the latter!

In [5]:
pipeline = [
    (ModelFairness(metrics), "MyModelFairness"),
    (Performance(metrics), "MyModelPerformance"),
]
lens = Lens(model=credo_model, assessment_data=credo_data, pipeline=pipeline)

2022-09-28 18:31:14,085 - lens - INFO - Evaluator ModelFairness added to pipeline. Dataset used: assessment_data. Sensitive feature: SEX
2022-09-28 18:31:14,283 - lens - INFO - fairness metric, equal_opportunity, unused by PerformanceModule
2022-09-28 18:31:14,305 - lens - INFO - Evaluator Performance added to pipeline. 


Above, each of the `tuples` in the `list` is in the form `(instantiated_evaluator, id)`.

In [6]:
# notice that Lens functions can be chained together
results = lens.run().get_results()

2022-09-28 18:31:14,322 - lens - INFO - Running evaluation for step: MyModelFairness
2022-09-28 18:31:14,346 - lens - INFO - Running evaluation for step: MyModelPerformance


Let's check that we have results for both of our evaluators.

In [7]:
results['MyModelFairness'][0]

,type,value,subtype
0,equal_opportunity,0.027686,fairness
1,precision_score,0.016322,parity
2,recall_score,0.027686,parity


In [8]:
results['MyModelPerformance'][0]

,type,value
0,precision_score,0.628081
1,recall_score,0.360172


## That's it!

That should get you up and running. Next steps include:

* Trying out other evaluators (they are all accessible via `credoai.evaluators`)
* Checking out our developer guide to better understand the Lens ecosystem and see how you can extend it.
* Exploring the Credo AI Governance Platform, which will connect AI assessments with customizable governance to support reporting, compliance, multi-stakeholder translation and more!

## In-Depth Overview

CredoAI Lens is the assessment framework component of the broader CredoAI suite.
It is usable as a standalone gateway to assessments or in combination
with CredoAI's Governance Platform. 

Understanding how your AI systems are operating is the most important step in intervening upon your system. From the technically complicated questions of improving a system to the business-relevant questions of whether to deploy a system - everything is fundamentally founded upon good observability. Lens strives to make assessment comprehensive, easy, and adaptable. The primary outputs from Lens are **assessment results** in the form of various metrics.

When used in combination with CredoAI's Governance Platform, Lens can also be used to provide `evidences` to the Governance object. See !insert link to notebook! for more details on any governance related operations.

### Evaluators

Evaluators are the core units that perform any type of assessment on models and/or datasets. They can be developed internally by CredoAI, as well leverage existing frameworks that are alreayd part of the broader open-source ecosystem. Cusom analytics can be also easily wrapped into an evaluator (see `developer guide ! insert link`).

AI system assessment starts with verifying standard performance metrics to an evolving set of assessments falling under the banner of *Responsible AI*. A non-exhaustive list includes

* Fairness
* Explainability
* Performance
* Robustness
* Equity
* Privacy
* Security

These different categories of assessment differ substantially based on whether one is 
evaluating datasets or models, what kind of model (e.g., tabular, NLP, computer vision), and the use-case. As the ecosystem develops, Lens will support assessing a broader range of AI systems. Currently, we are focused on Fairness.

### Governance

While Lens is a stand-alone assessment framework, its value is increased when combined with the CredoAI Governance Platform. The platform supports multi-stakeholder `Alignment` on how to assess your AI systems (e.g., what does good look like for this system?). 

It also supports translating a set of requirements, defined either within an organization or by existing legistlation, into a cohesive report that is accessible to a range of diverse stakeholders.

Check out the [Connecting with Governance App](https://credoai-lens.readthedocs.io/en/latest/notebooks/governance_integration.html) for information.



## Breaking Down The Steps

### Preparing artifacts

Lens interacts with Artifacts which wrap model and data objects and standardize them for use by different evaluators.

Below we create a `ClassifiactionModel` artifact. This is a light wrapper for any kind of fitted classification model-like object. 

We also create a `TabularData` artifact which stores X, y and sensitive features.

Both of these objects are customizable. See `lens_customization.ipynb` for more information.


In [9]:
# set up model and data artifacts
credo_model = ClassificationModel(name="credit_default_classifier", model_like=model)
credo_data = TabularData(
    name="UCI-credit-default",
    X=X_test,
    y=y_test,
    sensitive_features=sensitive_features_test,
)

#### Model

Model type objects, like `ClassificationModel` used above, serve as adapters between arbitrary models and the evaluators in Lens. Some evaluators depend on Model instantiating certain methods. For example, `ClassificationModel` can accept any generic object having `predict` and `predict_proba` methods, including fitted sklear pipelines. For more details see the source code: !!Insertlink!!


The user can make use of existing model artifacts or create their own. In order to do that, see deatails in !!insertlink!!
<br>



#### CredoData

_Data_ type artifact, like `TabularData` serve as adapters between datasets and the evaluators in Lens.

When you pass data to a _Data_ artifact, the artifact performs various steps of validation, and formats them so that they can be used by evaluators. The aim of this procedure is to preempt errors down the line.

You can pass Data to Lens as a **training dataset** or an **assessment dataset** (see lens class documentation). If the former, it will not be used to assess the model. Instead, dataset assessments will be performed on the dataset (e.g., fairness assessment). The validation dataset will be assessed in the same way, but _also_ used to assess the model, if provided.

Similarly to _Model_ type objects, _Data_ objects can be customized, see !!insertlink!!

### Evaluators 

Lens uses the above artifacts to ensure a successfull run of the evaluators. As we have seen in the sections [Lens in 5 minutes](##Lens-in-5-minutes) and [Adding multiple evaluators](##Adding-multiple-evaluators), multiple evaluators can be added to _Lens_ pipeline. Each evaluators contains information on what it needs in order to run successfully, and it executes a validation step at _add_ time.

The result of the validation depends on what artifacts are available, their content and the type of evaluator being added to the pipeline. In case the validation process fails, the user is notified the reason why the evaluator cannot be added to the pipeline.

See for example:

In [10]:
from credoai.evaluators import Privacy
lens.add(Privacy())

2022-09-28 18:31:14,477 - lens - INFO - Evaluator Privacy NOT added to the pipeline: Missing object training_data


Currently no automatic run of evaluators is supported. However, when Lens is used in combination with Credo AI Platform, it is possible to download an assessment plan which then gets converted into a set of evaluations that Lens can run programmatically. For more information: !!insert link!!

### Run Lens

After we have initialized _Lens_ the _Model_ and _Data_ (`ClassificationModel` and `TabularData` in our example) type artifacts, we can add whichever evaluators we want to the pipeline, and finally run it!

In [11]:
lens = Lens(model=credo_model, assessment_data=credo_data)
metrics = ['precision_score', 'recall_score', 'equal_opportunity']
lens.add(ModelFairness(metrics=metrics), id='MyModelFairness')
lens.run()

2022-09-28 18:31:14,745 - lens - INFO - Evaluator ModelFairness added to pipeline. Dataset used: assessment_data. Sensitive feature: SEX
2022-09-28 18:31:14,746 - lens - INFO - Running evaluation for step: MyModelFairness


As you can notice, when adding _evaluators_ to lens, they need to be instantiated. If any extra arguments need to be passed to the evaluator (like metrics in this case), this is the time to do it.

**Getting Assessment Results**

Afte the pipeline is run, the results become accessible via the method `get_results()`

`lens.get_results()` provides a dictionary where the results of the evaluators are stored as values, and the keys correspond to the ids of the evaluators.  

In the previous case we specified the id of the evaluator when we added `ModelFairness` to the pipeline, however `id` is an optional argument for the `add` method. If omitted, a random one will be generated.

In [12]:
lens.get_results()

{'MyModelFairness': [                type     value   subtype
  0  equal_opportunity  0.027686  fairness
  1    precision_score  0.016322    parity
  2       recall_score  0.027686    parity,
        SEX             type     value
  0  female  precision_score  0.618687
  1    male  precision_score  0.635009
  2  female     recall_score  0.344585
  3    male     recall_score  0.372271]}

**Credo AI Governance Platform**

For information on how to interact with the plaform, please look into: [Connecting with Governance App](https://credoai-lens.readthedocs.io/en/latest/notebooks/governance_integration.html) tutorial for directions.
